# `Associated Film Festivals Feature`

In [126]:
# Importing python librairies
import requests
import urllib.request, urllib.parse, urllib.error
import re
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime, timedelta

In [127]:
# Reading in master datasest and preping Account Created Date column to datetime object
df_master_dataset = pd.read_csv("../Master Datasets/Master_Dataset_Raw_SportsEvents.csv")

df_master_dataset['Account Created Date'] = pd.to_datetime(df_master_dataset['Account Created Date'])

df_master_dataset['Associated Film Festivals'] = pd.Series(dtype=str)

df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Date (No Timestamp),Interest,Sports Events,Associated Film Festivals
0,121169,STY1659426957,103,USR1606807023,female,34,Trend of the Year: Barbiecore,"The colour of the year is here, and it's *drum...",https://image.pixstory.com/Pixstory-image-1659...,2020-01-12 12:47:00,2020-01-12,"trends, fashion, barbie",Winter Youth Olympics,NaN
1,127727,STY1660634861,103,USR1606807023,female,34,Abomination of the day,We Indians do love to bastardise our foods- Ch...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,2020-01-12,"Food, momos, weird menus",Winter Youth Olympics,NaN
2,123665,STY1660027898,103,USR1606807023,female,34,Shameful headline in 2022,Can professors not have personal lives? \n\nAd...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,2020-01-12,"misogyny, st xaviers",Winter Youth Olympics,NaN
3,130517,STY1661151635,103,USR1606807023,female,34,Woman lawyer arrested for abusing security guard,"She was recorded on video manhandling him, sho...",https://image.pixstory.com/Pixstory-image-1661...,2020-01-12 12:47:00,2020-01-12,"Technology, History, Food, Entertainment, Spor...",Winter Youth Olympics,NaN
4,125466,STY1660372361,109,USR1606851217,others,31,What is the Inflation Reduction Act?,The House passed the Inflation Reduction Act o...,https://image.pixstory.com/Pixstory-image-1660...,2020-02-12 01:03:00,2020-02-12,us government,Six Nations,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,125349,STY1660325841,71665,USR1660325291,NaN,18,COLLEGE LIFE,MODR CLASS,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 22:58:00,2022-12-08,"college life, college life umang","FIFA World Cup, World Championships",NaN
94996,125345,STY1660325638,71673,USR1660325486,NaN,17,The one with the bestie😍,College life without a friend is useless. Enjo...,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:01:00,2022-12-08,college life,"FIFA World Cup, World Championships",NaN
94997,125355,STY1660326971,71697,USR1660326022,NaN,19,College,"Sheridan college, HazelMcCallion Canpus",https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,2022-12-08,"college life, college life umang","FIFA World Cup, World Championships",NaN
94998,125351,STY1660326276,71698,USR1660326026,NaN,21,A tale of friendship,Check this out.,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,2022-12-08,friends,"FIFA World Cup, World Championships",NaN


# Adding 2022 Film Festivals

https://www.film-fest-report.com/home/film-festivals-2022

In [128]:
# The URL of the webpage to be scraped is stored in the 'url' variable
url = 'https://www.film-fest-report.com/home/film-festivals-2022'

# The urllib.request.urlopen method retrieves the content of the webpage at the specified URL
# The read() method then reads the content into a string
html = urllib.request.urlopen(url).read()

# The BeautifulSoup constructor parses the HTML content and creates a BeautifulSoup object
# The 'html.parser' argument specifies the parser to be used
soup = BeautifulSoup(html, 'html.parser')

# The find_all() method is used to find all 'p' tags in the HTML content
p_tags = soup.find_all('p')

# The result of the find_all() method is a ResultSet object, which is converted to a list and stored in the same variable 'p_tags'
p_tags = list(p_tags)

In [129]:
# Print the text content of each <p> tag
film_festivals_list = []

# For every p_tag, grab the text as long as its not empty
for p_tag in p_tags:
    if p_tag.text != '':
        film_festivals_list.append(p_tag.text)

# Cleaning and removing non-film festivals from list
del film_festivals_list[0]
del film_festivals_list[0]
del film_festivals_list[118]
del film_festivals_list[118]
del film_festivals_list[118]
del film_festivals_list[118]
del film_festivals_list[34]
del film_festivals_list[62]

In [130]:
# Create a dictionary of the film festivals where the key is festival and the value is the dates
film_festivals_dict = {}
for i in range(0, len(film_festivals_list), 2):
    film_festivals_dict[film_festivals_list[i]] = film_festivals_list[i+1]

In [131]:
# Convert the dictionary to a DataFrame
df_2022 = pd.DataFrame(film_festivals_dict.items(), columns=['Festival Name', 'Raw Date'])

In [132]:
# Typecast dates to strings
df_2022['Start Date'] = pd.Series(dtype=str)
df_2022['End Date'] = pd.Series(dtype=str)

In [133]:
# Prepeing the dataframe by brending down the 
for index, row in df_2022.iterrows():
    # Parse the start and end dates from the raw date string
    start_date_str, end_date_str = row['Raw Date'].split("-")

    # Cleanign up date...
    if (row['Raw Date'][1] == "-" or row['Raw Date'][2] == "-"):
        start_date_str = start_date_str + " " + end_date_str.split(" ")[1] + " " + end_date_str.split(" ")[2]
    else:
        start_date_str = start_date_str + end_date_str.split(" ")[3]

    end_date_str = end_date_str.strip()

    # Convert the start and end dates to datetime objects
    start_date = datetime.strptime(start_date_str, "%d %B %Y")
    end_date = datetime.strptime(end_date_str, "%d %B %Y")

    # Add dates to df
    row['Start Date'] = start_date
    row['End Date'] = end_date

In [134]:
# Delete raw date column becuase its no longer needed
del df_2022["Raw Date"]

In [135]:
df_2022

,Festival Name,Start Date,End Date
0,Sundance Film Festival 2022,2022-01-20 00:00:00,2022-01-30 00:00:00
1,International Film Festival Rotterdam 2022,2022-01-26 00:00:00,2022-02-06 00:00:00
2,Clermont-Ferrand International Short Film Fest...,2022-01-28 00:00:00,2022-02-05 00:00:00
3,Berlin International Film Festival 2022,2022-02-10 00:00:00,2022-02-20 00:00:00
4,Glasgow International Film Festival 2022,2022-03-02 00:00:00,2022-03-13 00:00:00
5,Santa Barbara International Film Festival 2022,2022-03-02 00:00:00,2022-03-12 00:00:00
6,True/False Film Fest 2022,2022-03-03 00:00:00,2022-03-06 00:00:00
7,SXSW 2022,2022-03-11 00:00:00,2022-03-20 00:00:00
8,CPH:DOX 2022,2022-03-23 00:00:00,2022-04-03 00:00:00
9,Hong Kong International Film Festival 2022,2022-03-31 00:00:00,2022-04-11 00:00:00


# Adding 2020 Film Festivals

https://www.filmfestivaldatabase.com

In [136]:
# Read in CSV of raw film festivals for 2020
df_2020 = pd.read_csv("raw_film_festivals_data/2020 Film Festival Database & Calendar - 2020 DATABASE.csv")

In [137]:
df_2020

,Object ID,Last Year's Month,ADDRESS,Primary Category for Calendar,Category,GA,Festival Name,Website,FilmFreeway / Submissions,Date Changes & Cancellations due to COVID-19,...,Animation,Comedy,Student,Misc. Niche,"Environment, Outdoor, Adventure, Wildlife","Social Justice, Peace",Children's,Dance,Christian,"Experimental, Video Art (etc)"
0,765.0,NaN,"1300 North Atlantic Avenue, Cocoa Beach, Flori...",Short,Short,NaN,15 Minutes of Fame Indie Film Festival,https://etheridgeproductions.com/,https://filmfreeway.com/15MinutesofFame,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,935.0,11.0,"352 Cypress Street, Abilene, Texas 79601 Unite...",Short,Short,NaN,24fps International Short Film Festival,http://24fpsfest.com,https://filmfreeway.com/24fpsfest,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,769.0,NaN,"567 Colorado Avenue, Carbondale, Colorado 81623","Environment, Outdoor, Adventure, Wildlife","Short, Environmental/Outdoor/Adventure/Wildlife",NaN,5Point Adventure Film Festival,https://5pointfilm.org/,https://filmfreeway.com/5PointFilmFestival,New dates: Oct 14-18,...,NaN,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN
3,926.0,NaN,"1101 Center Avenue Mall, Fort Collins, Colorad...",Student,"Student, Short",NaN,ACT Human Rights Film Festival,https://actfilmfest.colostate.edu,https://filmfreeway.com/ACTHumanRightsFilmFest...,Postponed,...,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,11.0,"3545 Lipan Street Denver, CO 80211 United States",Misc. Niche,Misc. Niche,NaN,Action Figure Film Festival,http://www.actionfigurefilmfestival.com/,https://filmfreeway.com/afff,NaN,...,NaN,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1017,892.0,10.0,"Sutton, WV United States",General,General,NaN,WV FILMmakers Festival,http://wvfilmmakersfestival.org,https://filmfreeway.com/WVFILMmakersFestival,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1018,749.0,NaN,"Yale University New Haven, CT 06520 United States",Student,Student,NaN,Yale Student Film Festival,http://www.yalestudentfilmfest.com/,https://filmfreeway.com/YSFF,NaN,...,NaN,NaN,x,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1019,829.0,10.0,"328 Jackson St, Columbus, IN 47201",General,General,NaN,Yes Film Festival,https://www.yesfilmfestival.com/,https://filmfreeway.com/YESFilmFestival,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1020,866.0,11.0,"540 Nepperhan Avenue Suite 556 Yonkers, New Yo...",General,General,NaN,"YoFiFest, The Yonkers Film Festival",https://www.yofifest.com/,https://filmfreeway.com/YoFiFest,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [138]:
# Drop all columns besides these listed
df_2020 = df_2020.loc[:, ['Festival Name', 'Start Date', 'End Date']]

In [139]:
# select rows where 'Start Date' is 'tbd' and drop them
df_2020 = df_2020.drop(df_2020[df_2020['Start Date'] == 'tbd'].index)

In [140]:
# convert 'Start Date' and 'End Date' columns to datetime objects
df_2020['Start Date'] = pd.to_datetime(df_2020['Start Date'], format='%m/%d/%y')
df_2020['End Date'] = pd.to_datetime(df_2020['End Date'], format='%m/%d/%y')

In [141]:
df_2020

,Festival Name,Start Date,End Date
0,15 Minutes of Fame Indie Film Festival,2020-02-29,2020-02-29
1,24fps International Short Film Festival,2020-11-06,2020-11-07
2,5Point Adventure Film Festival,2020-04-24,2020-04-26
3,ACT Human Rights Film Festival,2020-04-03,2020-04-11
5,Action On Film Festival,2020-07-23,2020-08-02
...,...,...,...
1014,"Workers Unite Film Festival, The",2020-05-08,2020-05-23
1016,Worldfest-Houston International Film & Video F...,2020-04-17,2020-04-26
1018,Yale Student Film Festival,2020-03-27,2020-03-29
1020,"YoFiFest, The Yonkers Film Festival",2020-11-06,2020-11-13


# Adding 2021 Film Festivals

https://www.screendaily.com/news/2021-film-festivals-and-markets-latest-dates-postponements-and-cancellations/5155284.article

In [142]:
# Make a request to the web page
# url = 'https://www.screendaily.com/news/2021-film-festivals-and-markets-latest-dates-postponements-and-cancellations/5155284.article'
# html_bytes = urllib.request.urlopen(url).read()

# Read in raw HTML content from HTML file
with open('raw_film_festivals_data/2021_film_festivals_html.html', 'rb') as file:
    html_bytes = file.read()

# Parse the HTML content of the page
soup = BeautifulSoup(html_bytes, 'html.parser')

# Find all <p> tags
p_tags = soup.find_all('p')

# Created a list of p_tags
p_tags = list(p_tags)

# Remove the first 6 entired of the p_tags list becuase they are trash
p_tags = p_tags[6:len(p_tags)]

In [143]:
# Get p_tag text and add to list
film_festivals_list_raw = []
for tag in p_tags:
    film_festivals_list_raw.append(tag.text)

In [144]:
# convert raw film festivals list into dictiornary where key is the festival and value is the raw date
film_festivals_dict = {}
for fest in film_festivals_list_raw:
    fest = fest.split(",")
    film_festivals_dict[fest[0]] = fest[1]

In [145]:
# Convert the dictionary to a DataFrame
df_2021 = pd.DataFrame(film_festivals_dict.items(), columns=['Festival Name', 'Raw Date'])

In [146]:
# Create empty datetime series
df_2021['Start Date'] = pd.Series(dtype=str)
df_2021['End Date'] = pd.Series(dtype=str)

In [147]:
# locate and change a specific value
df_2021.loc[1, 'Raw Date'] = "Netherlands – June 2-6"

# delete a row and reset the index
df_2021 = df_2021.drop(42, axis=0).reset_index(drop=True)

In [148]:
df_2021

,Festival Name,Raw Date,Start Date,End Date
0,Goteborg Film Festival,Sweden - January 29-February 8,NaN,NaN
1,International Film Festival Rotterdam,Netherlands – June 2-6,NaN,NaN
2,SCAD aTVfest,US - February 4-6,NaN,NaN
3,Sundance Film Festival,US - January 28-February 3,NaN,NaN
4,Glasgow Film Festival,UK - February 24-March 7,NaN,NaN
...,...,...,...,...
115,French Film Festival,UK - November 3-December 10,NaN,NaN
116,Made In Prague Festival,UK - November 7-December 10,NaN,NaN
117,Ventana Sur,Argentina - November 29-December 3,NaN,NaN
118,Red Sea Film Festival,Saudi Arabia - December 6-15,NaN,NaN


In [149]:
# Convert raw date column into two datetime objects of start Date and end Date

for index, row in df_2021.iterrows():

    raw_date = re.split(r'-|–', row['Raw Date'])
    start_date = raw_date[1].strip()
    end_date =  raw_date[2].strip()

    month = start_date.split(" ")[0]

    if (len(end_date.split(" ")) < 2):
        end_date = month + " " + end_date

    start_date = start_date + " 2021"
    end_date = end_date + " 2021"

    df_2021.at[index, 'Start Date'] = start_date
    df_2021.at[index, 'End Date'] = end_date

In [150]:
# convert the string date column to datetime objects
df_2021['Start Date'] = pd.to_datetime(df_2021['Start Date'], format='%B %d %Y')
df_2021['End Date'] = pd.to_datetime(df_2021['End Date'], format='%B %d %Y')

# Drop the raw date column
df_2021 = df_2021.drop('Raw Date', axis=1)

In [151]:
df_2021

,Festival Name,Start Date,End Date
0,Goteborg Film Festival,2021-01-29,2021-02-08
1,International Film Festival Rotterdam,2021-06-02,2021-06-06
2,SCAD aTVfest,2021-02-04,2021-02-06
3,Sundance Film Festival,2021-01-28,2021-02-03
4,Glasgow Film Festival,2021-02-24,2021-03-07
...,...,...,...
115,French Film Festival,2021-11-03,2021-12-10
116,Made In Prague Festival,2021-11-07,2021-12-10
117,Ventana Sur,2021-11-29,2021-12-03
118,Red Sea Film Festival,2021-12-06,2021-12-15


In [152]:
# Concatenate the 3 data frames
df_film_festivals = pd.concat([df_2022, df_2021, df_2020])

# typecast column to datetime object
df_film_festivals['Start Date'] = pd.to_datetime(df_film_festivals['Start Date'])
df_film_festivals['End Date'] = pd.to_datetime(df_film_festivals['End Date'])
df_film_festivals = df_film_festivals.sort_values(by='Start Date', ascending=True)

# Print the concatenated data frame
df_film_festivals

,Festival Name,Start Date,End Date
815,San Luis Obispo Jewish Film Festival,2020-01-02,2020-01-05
684,Palm Springs International Film Festival,2020-01-02,2020-01-13
42,Arkansas Shorts,2020-01-04,2020-01-04
554,Miami Jewish Film Festival,2020-01-09,2020-01-20
235,Dance Camera West,2020-01-09,2020-01-12
...,...,...,...
53,Lübeck Nordic Film Days 2022,2022-11-02,2022-11-06
54,International Documentary Festival Amsterdam 2022,2022-11-09,2022-11-20
56,PÖFF Tallinn Black Nights Film Festival 2022,2022-11-11,2022-11-27
55,Norwich Film Festival 2022,2022-11-11,2022-11-20


# Merging festival names into master dataset

In [153]:
# Convert date columns to datetime objects
df_master_dataset["Account Created Date"] = pd.to_datetime(df_master_dataset["Account Created Date"])
df_film_festivals["Start Date"] = pd.to_datetime(df_film_festivals["Start Date"])
df_film_festivals["End Date"] = pd.to_datetime(df_film_festivals["End Date"])

# Create a list to store the associated film festivals for each post
associated_festivals = []

# Loop through each post
for index, row in df_master_dataset.iterrows():
    # Find all film festivals where the post date falls within the festival date range
    
    # filter the dataframe based on the given date
    filtered_df = df_film_festivals[(df_film_festivals['Start Date'] <= row["Account Created Date"]) & (df_film_festivals['End Date'] >= row["Account Created Date"])]

    # extract the film festival names as a list
    film_festivals = filtered_df['Festival Name'].tolist()
    str_film_festivals = ', '.join(film_festivals)

    # Add the relevant festivals to the associated_festivals list for this post
    associated_festivals.append(str_film_festivals)

# Add the associated_festivals list as a new column in df_posts
df_master_dataset["Associated Film Festivals"] = associated_festivals


In [154]:
df_master_dataset

,Story Primary ID,Story ID,User Primary ID,User ID,Gender,Age,Title,Narrative,Media,Account Created Date,Date (No Timestamp),Interest,Sports Events,Associated Film Festivals
0,121169,STY1659426957,103,USR1606807023,female,34,Trend of the Year: Barbiecore,"The colour of the year is here, and it's *drum...",https://image.pixstory.com/Pixstory-image-1659...,2020-01-12 12:47:00,2020-01-12,"trends, fashion, barbie",Winter Youth Olympics,"Palm Springs International Film Festival, Miam..."
1,127727,STY1660634861,103,USR1606807023,female,34,Abomination of the day,We Indians do love to bastardise our foods- Ch...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,2020-01-12,"Food, momos, weird menus",Winter Youth Olympics,"Palm Springs International Film Festival, Miam..."
2,123665,STY1660027898,103,USR1606807023,female,34,Shameful headline in 2022,Can professors not have personal lives? \n\nAd...,https://image.pixstory.com/Pixstory-image-1660...,2020-01-12 12:47:00,2020-01-12,"misogyny, st xaviers",Winter Youth Olympics,"Palm Springs International Film Festival, Miam..."
3,130517,STY1661151635,103,USR1606807023,female,34,Woman lawyer arrested for abusing security guard,"She was recorded on video manhandling him, sho...",https://image.pixstory.com/Pixstory-image-1661...,2020-01-12 12:47:00,2020-01-12,"Technology, History, Food, Entertainment, Spor...",Winter Youth Olympics,"Palm Springs International Film Festival, Miam..."
4,125466,STY1660372361,109,USR1606851217,others,31,What is the Inflation Reduction Act?,The House passed the Inflation Reduction Act o...,https://image.pixstory.com/Pixstory-image-1660...,2020-02-12 01:03:00,2020-02-12,us government,Six Nations,"New Jersey International Film Festival, Palm B..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94995,125349,STY1660325841,71665,USR1660325291,NaN,18,COLLEGE LIFE,MODR CLASS,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 22:58:00,2022-12-08,"college life, college life umang","FIFA World Cup, World Championships",
94996,125345,STY1660325638,71673,USR1660325486,NaN,17,The one with the bestie😍,College life without a friend is useless. Enjo...,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:01:00,2022-12-08,college life,"FIFA World Cup, World Championships",
94997,125355,STY1660326971,71697,USR1660326022,NaN,19,College,"Sheridan college, HazelMcCallion Canpus",https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,2022-12-08,"college life, college life umang","FIFA World Cup, World Championships",
94998,125351,STY1660326276,71698,USR1660326026,NaN,21,A tale of friendship,Check this out.,https://image.pixstory.com/Pixstory-image-1660...,2022-12-08 23:10:00,2022-12-08,friends,"FIFA World Cup, World Championships",


In [155]:
# save dataframe to csv file
df_master_dataset.to_csv('../Master Datasets/Master_Dataset_Raw_FilmFestivals.csv', index=False)